# Damped Lyman Alpha absorber system towards a QSO

Figure 9.7 from Chapter 9 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot spectrum of the damped Lyman $\alpha$ absorption (DLA) line from a cloud along the line of sight towards
QSO J021741.8-370100 ($z_{em}$=2.910).  Data are from the ESO UVES spectrograph obtained from the
[ESO Science Archive Facility](http://archive.eso.org).  The DLA of the QSO has a 
redshift of  $z_{abs}$=2.429 and a column density of $\log_{10} N_{HI}=$20.62$\pm$0.08.

The QSO was first identified as a DLA system by [Maza, J., Wischnjewsky, M., & Antezana, R. 1996, Rev. Mex. Astron. Astrophys., 32, 35](https://ui.adsabs.harvard.edu/abs/1996RMxAA..32...35M), and the column
density of the DLA was measured by
[Zafar et al., 2013, A&A 556, A140](https://ui.adsabs.harvard.edu/abs/2013A%26A...556A.140Z).

Data were selected and extracted from the ESO UVES database by Dr. Stephan Frank, who normalized the spectrum
in the region of interest and computed the damped Ly$\alpha$ absorption profile corresponding to the published
column density.  These data are provided in an external file associated with this notebook.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

# for smoothing the UVES spectrum 

from astropy.convolution import convolve, Box1DKernel

# suppress nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)
warnings.filterwarnings('ignore',category=RuntimeWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig9_7' 

# graphic aspect ratio = width/height

aspect = 2.5

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Spectral Data

Data are in a 3-column format, wavelength, relative flux units (F$_\lambda$), and flux error:
 * wave - wavelength in Angstroms, observed frame
 * flux_norm - continuum-normalized flux
 * model - absorption line model for the DLA
 
The redshift of the DLA is z_abs=2.429, which we use to transform the observed wavelenghts into rest wavelength.

In [ ]:
dataFile='J021741.8-370100_norm.txt'
z_abs = 2.429

data = pd.read_csv(dataFile,sep=r'\s+',comment='#')
lam = np.array(data['wave'])/(1+z_abs)
flux = np.array(data['flux_norm'])
fDLA = np.array(data['model']) 

# Smooth the spectrum with a boxcar (running mean) of width 7 pixels

width = 7
smFlux = convolve(flux, Box1DKernel(width))

# Plotting Limits

xMin = 1200
xMax = 1232

yMin = -0.05 # normalized flux
yMax = 1.15

## Make the Plot

Spectrum as a solid black step plot, damped Ly$\alpha$ absorption line model as a dashed black line.

In [ ]:
fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)

ax.set_xlim(xMin,xMax)
ax.set_ylim(yMin,yMax)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

ax.xaxis.set_major_locator(MultipleLocator(5.))
ax.xaxis.set_minor_locator(MultipleLocator(1.))
ax.set_xlabel(r'Wavelength [\AA]')
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_ylabel(r'Relative flux', fontsize=axisFontSize)

ax.plot(lam,smFlux,'-',ds='steps-mid',color='black',lw=0.5,zorder=10)

# Model DLA profile plus continuum (1) and zero flux (0) lines

ax.plot(lam,fDLA,'-',color='black',lw=1.0,zorder=8)
ax.hlines([0.0,1.0],xMin,xMax,ls=':',color='black',lw=0.5,zorder=5)

# make the plot

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')